In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pymc3 as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from scipy import stats, optimize
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import time
import pandas as pd

def data_form(train_data, test_data):
    '''This function takes in the training dataset and test dataset and delivers a '''

    #Define target columns
    y_columns = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ','GYSR', 'SOP4', 'RVSZ',
                 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X','K6QO', 'QBOL',
                 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

    # Remove rows with missing target
    train_data.dropna(axis=0, subset=y_columns, inplace=True)

    ##-------##### MJP - 11 Sept 2020
    #convert date using pandas
    train_data['join_date'] = pd.to_datetime(train_data['join_date'])
    test_data['join_date'] = pd.to_datetime(test_data['join_date'])

    #add age column (== birth_year)
    train_data['age'] = 2020 - train_data["birth_year"]
    test_data['age'] = 2020 - test_data["birth_year"]

    #add age_joined column (age of client when joined)
    train_data['age_join'] = train_data['join_date'].dt.year - train_data["birth_year"]
    test_data['age_join'] = test_data['join_date'].dt.year - test_data["birth_year"]

    #period_client (== join_date in years; duration)
    train_data['period_client'] = 2020 - train_data['join_date'].dt.year
    test_data['period_client'] = 2020 - test_data['join_date'].dt.year

    #use occupation_code as a category variable
    train_data["occupation_code"] = train_data["occupation_code"].astype('category')
    test_data["occupation_code"] = test_data["occupation_code"].astype('category')

    #Separate target from predictors
    y_train = train_data[y_columns]
    X_train = train_data.drop(y_columns, axis = 1)

    X_test = test_data.drop(y_columns, axis = 1)
    y_test = test_data[y_columns]

    # Select categorical columns and numerical columns
    categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]

    numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

    # Keep selected columns only
    my_cols = categorical_cols + numerical_cols
    X_train = X_train[my_cols].copy()
    X_test = X_test[my_cols].copy()

    return X_train, y_train, X_test, y_test, categorical_cols, numerical_cols

In [4]:
train_data = pd.read_csv("/content/drive/My Drive/Regression Challenge Shared Folder/Train.csv") 
test_data = pd.read_csv("/content/drive/My Drive/Regression Challenge Shared Folder/Test.csv")

X = train_data
y_columns = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ',
                 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']
y = test_data[y_columns]

# Data formatting
X_train, y_train, X_test, y_test, cat_cols, num_cols = data_form(train_data, test_data)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [6]:
# Preprocessing for numerical data
numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

# Preprocessing for categorical data

categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
        transformers=[('num', numeric_transformer, num_cols), ('cat', categorical_transformer, cat_cols)])

In [7]:
X_train = pd.DataFrame([preprocessor.fit(X_test)])

In [8]:
# Cast X_train into a Numpy array 
X_train = X_train.to_numpy()

In [9]:
#Generate Model
linear_model = pm.Model()

with linear_model: 
    # Priors for unknown model parameters    
    alpha = pm.Normal("alpha", mu=0,sd=10)
    betas = pm.Normal("betas", mu=0,#X_tr.mean(), 
                               sd=10, 
                               shape=X.shape[1])
    sigma = pm.HalfNormal("sigma", sd=1)

    # Likelihood (sampling distribution of observations)
    likelihood = pm.Normal("likelihood", mu=10, sd=sigma, observed=y_train)

    # Obtain starting values via Maximum A Posteriori Estimate
    map_estimate = pm.find_MAP(model=linear_model, fmin=optimize.fmin_powell)

    # Instantiate Sampler
    step = pm.NUTS(scaling=map_estimate)

    # MCMC
    trace = pm.sample(1000, step, start=map_estimate, progressbar=True)


/usr/local/lib/python3.6/dist-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
/usr/local/lib/python3.6/dist-packages/pymc3/tuning/starting.py:102: UserWarning: In future versions, set the optimization algorithm with a string. For example, use `method="L-BFGS-B"` instead of `fmin=sp.optimize.fmin_l_bfgs_b"`.
  warnings.warn('In future versions, set the optimization algorithm with a string. '
logp = -2.2705e+06:  28%|██▊       | 1422/5000 [00:07<00:18, 190.88it/s]

Optimization terminated successfully.
         Current function value: 2270510.062335
         Iterations: 2
         Function evaluations: 1428


logp = -2.2705e+06:  29%|██▊       | 1428/5000 [00:08<00:20, 177.58it/s]
Sequential sampling (2 chains in 1 job)
NUTS: [sigma, betas, alpha]
100%|██████████| 1500/1500 [03:11<00:00,  7.81it/s]


In [10]:
# Prediction
ppc = pm.sample_ppc(trace, model=linear_model, samples=1000)

#What's the shape of this? 
list(ppc.items())[0][1].shape 

#Looks like I need to transpose it to get X_test samples on rows and posterior distribution samples on cols

for idx in [0,1,2,3,4,5]:
    predicted_yi = list(ppc.items())[0][1].T[idx].mean()
    print(predicted_yi)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: sample_ppc() is deprecated.  Please use sample_posterior_predictive()
  
100%|██████████| 1000/1000 [00:29<00:00, 34.06it/s]


10.000056702824441
10.002303871922619
9.99825876227941
10.002650221746954
9.999488598077178
10.000254815859478
